Helper Functions for Later.




In [253]:
def extract_dates(df,column):
    new_column = []
    for dd in df[column]:
        if type(dd) == dict:
            new_dd = math.trunc(dd['$date']/1000)
            new_dd = pd.to_datetime(new_dd, unit='s')
            new_column.append(new_dd)
        else:
            new_column.append(dd)
    df[column] = new_column
    return df

Let's first process and clean up all the tables. Let me start with the simplest one: ``users``.

Just looking at the first five rows you can tell four are identical.

Let me get rid of all the duplicate users, so you're left with only 212 of the original 495 (212/495 = 42.8%)

Another strange thing is that almost all of the data is from Wisconsin. Should ask the client about this.



In [281]:
import json
import pandas as pd
import gzip
import math

#Create Dataframe and clean up
users_df = pd.read_json('users.json', lines=True)
users_df['_id'] = users_df['_id'].apply(lambda x: x['$oid']) #extract _id
users_df = extract_dates(users_df, 'createdDate')
users_df = extract_dates(users_df, 'lastLogin')
#Rename Column
users_df = users_df.rename(columns={'_id':'userId'})
#Reorder Columns according to description
users_df = users_df[['userId', 'state', 'createdDate', 'lastLogin', 'role', 'active']]


#Remove duplicates
users_df.shape #495 entries
users_df['userId'].nunique() #212 unique user_id's.
users_df = users_df.drop_duplicates(subset='userId') #only 212 uniques left now.

#Check out geographical distribution
#users_df['state'].hist()

users_df.head()




,userId,state,createdDate,lastLogin,role,active
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,True
3,5ff1e1eacfcf6c399c274ae6,WI,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,True
6,5ff1e1e8cfcf6c399c274ad9,WI,2021-01-03 15:25:28,2021-01-03 15:25:28,consumer,True
7,5ff1e1b7cfcf6c399c274a5a,WI,2021-01-03 15:24:39,2021-01-03 15:24:39,consumer,True
9,5ff1e1f1cfcf6c399c274b0b,WI,2021-01-03 15:25:37,2021-01-03 15:25:37,consumer,True


Now let's move on to the next simplest table: ``brands``. No comment as of yet.  


---



In [280]:
#Create dataframe and clean up
brands_data = [json.loads(line) for line in gzip.open("brands.json.gz", "r")]
brands_df = pd.json_normalize(brands_data)
brands_df = brands_df.rename(columns={'_id.$oid': 'brandId', 'cpg.$id.$oid': 'cpg_id', 'cpg.$ref': 'cpg_ref'})
brands_df.head()

#Reorder Columns according to description
brands_df = brands_df[['brandId', 'barcode', 'brandCode', 'category', 'categoryCode', 'cpg_id', 'cpg_ref', 'topBrand', 'name']]

brands_df.head()



,brandId,barcode,brandCode,category,categoryCode,cpg_id,cpg_ref,topBrand,name
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,False,test brand @1612366101024
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,False,Starbucks
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146176
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146051
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,False,test brand @1612366146827


We investigate the receipts dataset. There are 1119 receipts.

We look at how much data is missing, in particular ``totalSpent``, ``rewardsReceiptItemList`` and ``purchasedItemCount``.

This says that 489/1119 = 43.7% of the data is missing these three fields, which will make it very hard to answer questions about spend and total number of transactions. I make the decision to completely drop these records, leaving us with 630 receipts.

Now we have to do something major. rec_df['rewardsReceiptItemList'] is itself JSON. I have to unpack this.





In [282]:
#Create dataframe and clean up
receipts_data = [json.loads(line) for line in gzip.open("receipts.json.gz", "r")]
receipts_df = pd.DataFrame(receipts_data)
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid']) #extract _id

receipts_df = extract_dates(receipts_df, 'createDate')
receipts_df = extract_dates(receipts_df, 'dateScanned')
receipts_df = extract_dates(receipts_df, 'finishedDate')
receipts_df = extract_dates(receipts_df, 'modifyDate')
receipts_df = extract_dates(receipts_df, 'pointsAwardedDate')
receipts_df = extract_dates(receipts_df, 'purchaseDate')

receipts_df.dtypes
receipts_df = receipts_df.astype({'pointsEarned': 'float64',
                                            'purchasedItemCount': 'Int64',
                                            'totalSpent': 'float64'})

receipts_df = receipts_df.rename(columns={'_id':'receiptId'})

#EDA
receipts_df.receiptId.count() #1119 receipts

receipts_df.totalSpent.isna().sum() #435 NaNs in totalSpent
receipts_df.rewardsReceiptItemList.isna().sum() #440 NaNs in rewardsReceiptItemList
receipts_df.purchasedItemCount.isna().sum() #484 NaNs in purchased Item Count
receipts_df[['totalSpent','rewardsReceiptItemList', 'purchasedItemCount']].isna().all(axis=1).sum() #435 NaNs

#Drop all rows missing these three fields
receipts_df = receipts_df.dropna(subset=['totalSpent']).copy()
receipts_df = receipts_df.dropna(subset=['rewardsReceiptItemList']).copy()
receipts_df = receipts_df.dropna(subset=['purchasedItemCount']).copy()

#Count of remaining
receipts_df.receiptId.count() #630 receipts.

receipts_df.head()


,0
receiptId,object
bonusPointsEarned,float64
bonusPointsEarnedReason,object
createDate,datetime64[ns]
dateScanned,datetime64[ns]
finishedDate,datetime64[ns]
modifyDate,datetime64[ns]
pointsAwardedDate,datetime64[ns]
pointsEarned,float64
purchaseDate,datetime64[ns]


rec_df['rewardsReceiptItemList'] gets reformatted here. Each receipt has its own item lists, and these are the item lists.

In [284]:
order_item_list_data = [i for i in receipts_df.rewardsReceiptItemList]
order_items_list = []
for order in order_item_list_data:
    for item in order:
        order_items_list.append(item)

items_df = pd.DataFrame(order_items_list)
items_df.head()
items_df.dtypes

,0
barcode,object
description,object
finalPrice,object
itemPrice,object
needsFetchReview,object
partnerItemId,object
preventTargetGapPoints,object
quantityPurchased,float64
userFlaggedBarcode,object
userFlaggedNewItem,object


So now we have our four tables.




In [285]:
#Now we have to do SQL, which we can directly do in the ipynb notebook (this is new to me).
!pip install pandasql
from pandasql import sqldf
import pandas as pd

#Do this so SQL will work
receipts_df = receipts_df.astype({'rewardsReceiptItemList': 'string'})

print(sqldf('''SELECT max(dateScanned)
FROM rec_df
LIMIT 10''')) #Latest Date is 2021-03-01 21:32:28.000000


print(sqldf('''
SELECT  brand_name,  DENSE_RANK () OVER (ORDER BY times_bought DESC) AS brand_rank
FROM (SELECT name brand_name, COUNT(*) times_bought FROM brands_df
      WHERE(
                SELECT receiptId FROM receipts_df
                WHERE dateScanned < '2021-03-01'
                AND dateScanned >= '2021-02-01'
             )
      GROUP BY name)
LIMIT 5;
'''))

print(sqldf('''
SELECT  brand_name,  DENSE_RANK () OVER (ORDER BY times_bought DESC) AS brand_rank
FROM (SELECT name brand_name, COUNT(*) times_bought FROM brands_df
      WHERE(
                SELECT receiptId FROM receipts_df
                WHERE dateScanned < '2021-02-01'
                AND dateScanned >= '2021-01-01'
             )
      GROUP BY name)
LIMIT 5;
'''))


print(sqldf('''
SELECT rewardsReceiptStatus, round(AVG(totalSpent), 2) as average_spend
FROM receipts_df
GROUP BY rewardsReceiptStatus
ORDER BY average_spend DESC;
'''))

print(sqldf('''
SELECT rewardsReceiptStatus, SUM(purchasedItemCount) as total_number_of_items
FROM receipts_df
GROUP BY rewardsReceiptStatus
ORDER BY total_number_of_items DESC;
'''))

print(sqldf('''
SELECT rewardsReceiptStatus, SUM(purchasedItemCount) as total_number_of_items
FROM receipts_df
GROUP BY rewardsReceiptStatus
ORDER BY total_number_of_items DESC;
'''))

print(sqldf('''
SELECT brands_df.name, round(SUM(receipts_df.totalSpent), 2) as total_spend
FROM users_df
INNER JOIN receipts_df ON users_df.user_id = receipts_df.userId
INNER JOIN items_df ON receipts_df.receipt_id = items_df.receipt_id
INNER JOIN brands_df ON items_df.barcode = brands_df.barcode
WHERE users_df.createdDate  < '2021-02-01'
AND users_df.createdDate >= '2020-08-01'
GROUP BY 1
ORDER BY 2 DESC
'''))

print(sqldf('''
SELECT brands_df.name, COUNT(receipts_df.totalSpent) as total_transactions
FROM users_df
INNER JOIN receipts_df ON users_df.user_id = receipts_df.userId
INNER JOIN items_df ON receipts_df.receipt_id = items_df.receipt_id
INNER JOIN brands_df ON items_df.barcode = brands_df.barcode
WHERE users_df.createdDate  < '2021-02-01'
AND users_df.createdDate >= '2020-08-01'
GROUP BY 1
ORDER BY 2 DESC
'''))


0

             max(dateScanned)
0  2021-03-01 21:32:28.000000
                 brand_name  brand_rank
0                 Baken-Ets           1
1              Caleb's Kola           1
2  Diabetic Living Magazine           1
3       Dippin Dots® Cereal           1
4           Health Magazine           1
                 brand_name  brand_rank
0                 Baken-Ets           1
1              Caleb's Kola           1
2  Diabetic Living Magazine           1
3       Dippin Dots® Cereal           1
4           Health Magazine           1
  rewardsReceiptStatus  average_spend
0              FLAGGED         180.45
1             FINISHED          81.17
2             REJECTED          24.36
  rewardsReceiptStatus  total_number_of_items
0             FINISHED                   8184
1              FLAGGED                   1014
2             REJECTED                    173


0